In [4]:
import sys

In [5]:
from pytket import Circuit
from tket.circuit import Tk2Circuit
# from tket.protocol import CircuitReplacer

In [6]:
sys.path.append("../../../tket2-rewriting/tket2_rewriting")
from circuit_generation.gadget import CXGadgetBenchmarkCircuit,SimpleLabelFriendlyCircuit
from pytket.circuit.display import render_circuit_jupyter
import numpy as np
from pytket.passes import DecomposeBoxes, AutoRebase
from pytket import OpType

# # Original circuit to be compiled
# orig_circuit = SimpleLabelFriendlyCircuit(width=5, depth=10, rng=np.random.default_rng(0))
# orig_circuit.n_2qb_gates()

In [7]:
orig_circuit = Circuit(2,2).CX(0,1).X(0).CX(0,1)

In [8]:
orig_circuit

[CX q[0], q[1]; X q[0]; CX q[0], q[1]; ]

In [9]:
sys.path.append("../../../tket2-rewriting/tket2_rewriting")

from cx_gadget.cx_gadget import CXGadget
from zzphase_flip.zzphase_flip import ZZPhaseFlip
from three_qubit_squash.three_qubit_squash_gadget import Squash3Gadget
from KAK.kak_gadget import KAKGadget
from label.label_gadget import LabelGadget

In [10]:
circuit_tk2 = Tk2Circuit(orig_circuit)
circuit_tk2

## Existing gadgets

In [11]:
from tket.optimiser import BadgerOptimiser
from time import time
from pytket.passes import GreedyPauliSimp, FullPeepholeOptimise

circuit = orig_circuit.copy()

opt = BadgerOptimiser(
    [
        # ZZPhaseFlip(),
        # CXGadget(),
        # Squash3Gadget(),
        # KAKGadget()
        LabelGadget()
    ]
)

In [12]:
# Optimise circuit and time the optimisation - this is doing something
start_time = time()
opt_circ = opt.optimise(circuit_tk2)
print(f"time taken: {time() - start_time}")


[DEBUG] get_map PID=41483 TID=8688410944 label_map_id=4618912832 map={}
[DEBUG] init_label_map PID=41483 TID=8688410944 label_map_id=4588982784 map={0: '0', 1: '0'}
[DEBUG] get_label q=0 -> 0 PID=41483 TID=8688410944 label_map_id=4588982784 map={0: '0', 1: '0'}
[DEBUG] get_label q=1 -> 0 PID=41483 TID=8688410944 label_map_id=4588982784 map={0: '0', 1: '0'}
[DEBUG]: I recognise a CX
[DEBUG] get_map PID=41483 TID=8688410944 label_map_id=4588982784 map={0: '0', 1: '0'}
[DEBUG] get_label q=0 -> 0 PID=41483 TID=8688410944 label_map_id=4588982784 map={0: '0', 1: '0'}
[DEBUG]: I recognise a X
[DEBUG] update_label q=0 -> 1 PID=41483 TID=8688410944 label_map_id=4588982784 map={0: '1', 1: '0'}
[DEBUG] get_map PID=41483 TID=8688410944 label_map_id=4588982784 map={0: '1', 1: '0'}
[DEBUG] get_label q=0 -> 1 PID=41483 TID=8688410944 label_map_id=4588982784 map={0: '1', 1: '0'}
[DEBUG] get_label q=1 -> 0 PID=41483 TID=8688410944 label_map_id=4588982784 map={0: '1', 1: '0'}
[DEBUG]: I recognise a CX
[

1 qubit gate, if CX : 0->1, 1->0, if I: 0->0, 1->1, otherwise ?
2 qubit gate, if CX: control 0 -> target stays as is, control 1 -> target swaps 0->1 or 1->0, control label is unchanged
else: control 0 -> target stays as is, control 1 -> target ?, control label is unchanged

In [13]:
opt_circ1 = opt_circ.to_tket1()
type(opt_circ1) , type(circuit)

(pytket._tket.circuit.Circuit, pytket._tket.circuit.Circuit)

In [14]:
print(f"Two qubit gates: {opt_circ1.n_2qb_gates()} vs previous: {circuit.n_2qb_gates()}")

Two qubit gates: 1 vs previous: 2


In [15]:
orig_circuit

[CX q[0], q[1]; X q[0]; CX q[0], q[1]; ]

In [16]:
opt_circ1

[X q[0]; CX q[0], q[1]; ]

In [17]:
# Follow with FullPeepholeOptimise
FullPeepholeOptimise().apply(opt_circ1)
print(f"Two qubit gates: {opt_circ1.n_2qb_gates()}")

Two qubit gates: 1


In [18]:
opt_circ1

[TK1(0, 1, 0) q[0]; CX q[0], q[1]; ]

In [19]:
# With GreedyPauliSimp the result is not as good
circuit = orig_circuit.copy()
GreedyPauliSimp().apply(circuit)
print(f"Two qubit gates: {circuit.n_2qb_gates()}")
# render_circuit_jupyter(circuit)

Two qubit gates: 0


In [20]:
opt_circ1

[TK1(0, 1, 0) q[0]; CX q[0], q[1]; ]

In [21]:
# With FullPeephole the result is not as good
circuit = orig_circuit.copy()
FullPeepholeOptimise().apply(circuit)
print(f"Two qubit gates: {circuit.n_2qb_gates()}")
# render_circuit_jupyter(circuit)

Two qubit gates: 0


In [22]:
# With both the result is not as good
circuit = orig_circuit.copy()
GreedyPauliSimp().apply(circuit)
FullPeepholeOptimise().apply(circuit)
print(f"Two qubit gates: {circuit.n_2qb_gates()}")
# render_circuit_jupyter(circuit)

Two qubit gates: 0
